# Streaming platformokon lévő sorozatok adatait vetettük össze az IMDb értékeléseikkel, majd szám, értékelés, és kor alapján vizsgáltuk őket.

Készítette:
<br>
Frey Ferenc Gábor
<br>
Varga Dániel István
<br>
Budai Sándor

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#import pandas
import pandas as pd

# **Adatbázisok betöltése és felesleges oszlopok törlése**

Az adatbázisokat panda dataframe-ként (adatkeretként) olvassuk be és később egyesítjük őket


A **titles** az IMDb-n lévő címeket tartalmazza
<br>
A **ratings** a **titles** kiegészítése értékelésekkel
<br>
A **shows** tartalmazza a streaming szolgáltatásokon lévő sorozatokat

In [ ]:
#Az adatbázisokat panda dataframe-ként olvassuk be
titles = pd.read_csv('/kaggle/input/imdb-dataset/title.basics.tsv.gz', compression='gzip', header=0, sep='\t', quotechar='"',
                      dtype={'tconst': 'string', 'titleType': 'string', 'primaryTitle': 'string', 'originalTitle': 'string', 'isAdult': 'int', 'startYear': 'string', 'endYear': 'string', 'runtimeMinutes': 'string', 'genres': 'string'})
del titles['isAdult']

ratings = pd.read_csv('/kaggle/input/imdb-dataset/title.ratings.tsv.gz', compression='gzip', header=0, sep='\t', quotechar='"',
                      dtype={'tconst': 'string', 'averageRating': 'double', 'numVotes': 'int'})
del ratings['averageRating']

shows = pd.read_csv('/kaggle/input/tv-shows-on-netflix-prime-video-hulu-and-disney/tv_shows.csv')
del shows['Unnamed: 0']
del shows['Rotten Tomatoes']
del shows['type']

In [ ]:
#1. lépés a StreamPlatform oszlop létrehozásához, hogy érdekesebb plotokat tudjunk csinálni ggplottal
#shows.loc[shows['Netflix'] == 1, 'Netflix'] = 'Netflix'
#shows.loc[shows['Hulu'] == 1, 'Hulu'] = 'Hulu'
#shows.loc[shows['Prime Video'] == 1, 'Prime Video'] = 'Prime Video'
#shows.loc[shows['Disney+'] == 1, 'Disney+'] = 'Disney+'
#merge then into 1 column -> streamPlatform
#shows.head(50)

**Címek (titles) egyesítése az értékelésekkel (ratings) a 'tconst' oszlop alapján**

A .merge metódus alapértelmezetten egy közös oszlop alapján egyesíti az adatokat

In [ ]:
title_ratings = pd.merge(titles, ratings)
title_ratings.head()

**Az összes olyan rekord törlése aminek a típusa NEM movie, tvMovie, tvSeries vagy tvMiniSeries**

In [ ]:
title_ratings_filtered = title_ratings.drop(title_ratings[
                  (title_ratings.titleType == 'tvEpisode') |
                  (title_ratings.titleType == 'tvShort') |
                  (title_ratings.titleType == 'tvSpecial') |
                  (title_ratings.titleType == 'short') |
                  (title_ratings.titleType == 'video') |
                  (title_ratings.titleType == 'videoGame')].index)
#display
title_ratings_filtered.tail()

**Cseréljük az összes \N értéket NaN értékekkel**

In [ ]:
title_ratings_filtered = title_ratings_filtered.replace('\\N',np.NaN)
title_ratings_filtered.head()

**Kombináljuk az imdb adatkeretet a streaming szolgáltatások adatait tartalmazóval és megtisztítjuk a végső adatkeretet**

In [ ]:
#imdb titles egyesítése a streamingel a címeik alapján
#merge titles from imdb (title_ratings_filtered) with titles on streaming services (shows) based on their title
filtered_df = pd.merge(title_ratings_filtered, shows, left_on='primaryTitle', right_on='Title')

#felesleges oszlopok törlése
#drop unwatned columns
filtered_df = filtered_df.drop(columns=['Title', 'Year'])

#minden sor kitörlése ami nem sorozat
#drop media that isn't tvSeries
filtered_df = filtered_df.drop(filtered_df[filtered_df.titleType != 'tvSeries'].index, axis=0)

#imdb értékelés nélküli sorok törlése
#drop titles without imdb rating
#filtered_df = filtered_df.drop(filtered_df[filtered_df.IMDb.isnull()].index, axis=0)

#ismétlödő sorok törlése imdb azonosító alapján
#drop duplicated titles based on tconst
filtered_df = filtered_df.drop_duplicates(subset=['tconst'], keep='last')

#címek törlése 100-nál kevesebb értékeléssel
#drop titles with less than x imdb ratings
#filtered_df = filtered_df.drop(filtered_df[filtered_df.numVotes <= 100].index, axis=0)

#címek törlése amik 1960 előtt jelentek még és 10000-nél kevesebb értékelésük van
#drop titles released before 1960 and has less than 10000 votes
filtered_df = filtered_df.drop(filtered_df[(filtered_df.startYear.astype(int) <= 1960) & (filtered_df.numVotes <= 10000)].index, axis=0)

#display
filtered_df

Ismert hiba: Az analízis alapjául szolgáló adatbázis (imdb-database) nem tartalmazza a másik adatbázis (tv-shows-on-netflix-prime...) összes elemét, ezért lehet olyan sorozat ami nem kerül be a végső adatkeretbe.

Emellett több népszerűtlen sorozatot is droppoltunk a végső adatkeretből, így lett 10 híján 1500-zal kevesebb (4121) rekordunk a tisztítás végén.

In [ ]:
#shows adatkeret (sorai, olszopai)
shows.shape

In [ ]:
filtered_df[(filtered_df.primaryTitle == 'Oliver Twist')]

Sorozatok IMDb értékelés nélkül, több ezer szavazattal

In [ ]:
filtered_df[(filtered_df.IMDb.isnull()) & (filtered_df.numVotes >= 3000)]

# **Vizualizálás**

In [ ]:
#correlation matrix
plt.figure(figsize=(16,12))
plt.title("correlation matrix",size=30)
sns.heatmap(filtered_df.corr(),cmap="YlOrRd",square=True,annot=True)

In [ ]:
#Colors representing the streaming services main colors in hex code
colors = ['#ed2f39','#62c462','#ffb84d','#4687fa']

In [ ]:
piedata = filtered_df.sum()['Netflix':"Disney+"]
plt.figure(figsize=(12,8))
plt.title("Sorozatok számának %-os eloszlása",size=30)
plt.pie(piedata,labels=piedata.index,colors=colors, autopct='%1.1f%%')
plt.legend(title='Streamer', bbox_to_anchor=(1, 0, 0.7, 1),loc='center right',)

Netflix (8.99 dollars/month)
<br>
Disney Plus (6.99 dollars/month)
<br>
Hulu (5.99 dollars/month)
<br>
Amazon Prime Video (12.99 dollars/month)

**??**

In [ ]:
#Who is the platform with the most highly rated shows?
#top_shows = shows['IMDb'] > 8
#top_shows.head(100)
#top_rated_shows = shows
#top_rated_shows.insert(9, "IMDB 8+", top_shows)

#indexNames = top_rated_shows[ top_rated_shows['IMDB 8+'] == 'False' ].index
# Delete these row indexes from dataFrame
#top_rated_shows.drop(indexNames , inplace=True)
#top_rated_shows.head(100)
#piedata = top_shows.sum()['Netflix':"Disney+"]
#plt.figure(figsize=(12,8))
#plt.pie(piedata,labels=piedata.index,autopct='%1.1f%%')
#plt.legend(title='Streamer', bbox_to_anchor=(1, 0, 0.7, 1),loc='center right',)
#top_shows = top_shows.set_index('Netflix':"Disney+")
#display(top_shows)

In [ ]:
#filtered_df.groupby(by=' IMDb').sum()[['Netflix','Hulu',"Disney+",'Prime Video']].plot(kind='bar',figsize=(12,8),style='bmh' ).title("correlation matrix",size=30)

In [ ]:
filtered_df.groupby(by='IMDb').sum()[['Netflix']].plot(kind='bar',figsize=(14,4),style='bmh',title=("Netflix tartalmak imdb pontszám szerinti eloszlása"))
filtered_df.groupby(by='IMDb').sum()[['Hulu']].plot(kind='bar',figsize=(14,4),style='bmh',title=("Hulu tartalmak imdb pontszám szerinti eloszlása"))
filtered_df.groupby(by='IMDb').sum()[['Disney+']].plot(kind='bar',figsize=(14,4),style='bmh',title=("Disney+ tartalmak imdb pontszám szerinti eloszlása"))
filtered_df.groupby(by='IMDb').sum()[['Prime Video']].plot(kind='bar',figsize=(14,4),style='bmh',title=("Prime Video tartalmak imdb pontszám szerinti eloszlása"))

In [ ]:
filtered_df.groupby(by='IMDb').sum()[['Netflix','Hulu','Disney+','Prime Video']].plot(kind='bar',figsize=(14,10),style='g',title=("Minden tartalom imdb pontszám szerinti eloszlása"))

In [ ]:
filtered_df.groupby(by='startYear').sum()[['Netflix']].plot(kind='bar',figsize=(14,4),style='bmh',title=("Netflix tartalmak év szerinti eloszlása"))
filtered_df.groupby(by='startYear').sum()[['Hulu']].plot(kind='bar',figsize=(14,4),style='bmh',title=("Hulu tartalmak év szerinti eloszlása"))
filtered_df.groupby(by='startYear').sum()[['Disney+']].plot(kind='bar',figsize=(14,4),style='bmh',title=("Disney+ tartalmak év szerinti eloszlása"))
filtered_df.groupby(by='startYear').sum()[['Prime Video']].plot(kind='bar',figsize=(14,4),style='bmh',title=("Prime Video tartalmak év szerinti eloszlása"))

In [ ]:
filtered_df.groupby(by='startYear').sum()[['Netflix','Hulu','Disney+','Prime Video']].plot(kind='bar',figsize=(14,10),style='g',title=("Minden tartalom év szerinti eloszlása"))

In [ ]:
filtered_df.groupby(by='Age').sum()[['Netflix']].plot(kind='bar',figsize=(14,4),style='bmh',title=("Netflix tartalmak korosztály szerinti eloszlása"))
filtered_df.groupby(by='Age').sum()[['Hulu']].plot(kind='bar',figsize=(14,4),style='bmh',title=("Hulu tartalmak korosztály szerinti eloszlása"))
filtered_df.groupby(by='Age').sum()[['Disney+']].plot(kind='bar',figsize=(14,4),style='bmh',title=("Disney+ tartalmak korosztály szerinti eloszlása"))
filtered_df.groupby(by='Age').sum()[['Prime Video']].plot(kind='bar',figsize=(14,4),style='bmh',title=("Prime Video tartalmak korosztály szerinti eloszlása"))

In [ ]:
filtered_df.groupby(by='Age').sum()[['Netflix','Hulu',"Disney+",'Prime Video']].plot(kind='bar',figsize=(12,10),style='bmh',title=("Minden tartalom korosztály szerinti eloszlása"))

In [ ]:
#sns.regplot(x='startYear',y='IMDb',data=filtered_df)